In [1]:
import os
os.chdir('..')

In [2]:
from team_functions import *

In [3]:
today = date.today()
print(f'Today is: {today}')

Today is: 2024-02-23


In [4]:
last_games = pd.read_excel('Predictions/games_odds.xlsx', 'Tonight')
prev_games = pd.read_excel('Predictions/games_odds.xlsx', 'Previous')

In [5]:
options = Options()
options.add_argument('--headless')
driver = Firefox(options=options)

In [6]:
url = 'https://www.basketball-reference.com/boxscores/'
soup = getSoup(url, driver=driver)

In [7]:
summaries = soup.find('div', {'class': 'game_summaries'})
games = summaries.findAll('table', {'class': 'teams'})

games_df = []
for idx, game in enumerate(games):
    loser = game.find('tr', {'class': 'loser'}).find('td').find('a').get('href').split('/')[2]
    winner = game.find('tr', {'class': 'winner'}).find('td').find('a').get('href').split('/')[2]
    
    game_row = {'Winner': winner, 'Loser': loser}
    games_df.append(game_row)
    
games_df = pd.DataFrame(games_df)

In [8]:
results = []
for team in last_games['visitor']:
    if team in list(games_df['Winner']):
        results.append(1)
    elif team in list(games_df['Loser']):
        results.append(0)
    else:
        print(f'Error checking results of team: {team}')

last_games['Results'] = np.array(results)

In [9]:
last_games

,date,visitor,local,v_line,l_line,Results
0,2024-02-16,NYK,PHI,1.952381,1.869565,1
1,2024-02-16,DET,IND,5.400000,1.166667,0
2,2024-02-16,ORL,CLE,3.600000,1.307692,1
3,2024-02-16,BRK,TOR,1.952381,1.869565,0
4,2024-02-16,PHO,DAL,2.100000,1.769231,0
5,2024-02-16,LAC,OKC,2.140000,1.740741,0
6,2024-02-16,BOS,CHI,1.359712,3.250000,1
7,2024-02-16,HOU,NOP,3.250000,1.359712,0
8,2024-02-16,CHO,UTA,4.400000,1.227273,1
9,2024-02-16,WAS,DEN,9.500000,1.068966,0


In [10]:
prev_games.tail()

,date,visitor,local,v_line,l_line,Results
77,2024-02-14,LAL,UTA,2.700000,1.487805,1
78,2024-02-14,LAC,GSW,2.200000,1.704225,1
79,2024-02-15,MIL,MEM,1.142857,6.000000,0
80,2024-02-15,GSW,UTA,1.800000,2.050000,1
81,2024-02-15,MIN,POR,1.235294,4.300000,1


In [11]:
prev_games = pd.concat([prev_games, last_games])

In [12]:
url = 'https://sportsbook.draftkings.com/leagues/basketball/nba'

In [13]:
driver.get(url)

html = driver.page_source
soup = bs(html, 'lxml')

driver.close()

In [14]:
def american_to_decimal(american):
    '''
    This functions transform an american system odd (e.g. -120) to a decimal system (e.g 1.8)
    
    INPUTS:
    player: row containing player stats
    
    OUTPUT:
    new_player: copy of the player df input with the updated Odd value in Decimal System
    '''
    # Check if the value is positive or negative (e.g. Odd = +100 or Odd = -120)
    # Transform it to decimal with the given formula
    if american > 0:
        decimal = (american/100) + 1
    else:
        decimal = (100/abs(american)) + 1
    
    return decimal

In [15]:
tables = soup.findAll('tbody', {'class': 'sportsbook-table__body'})
rows = []
for table in tables:
    for row in table.findAll('tr'):
        rows.append(row)
games, game, idx = [], {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}, 0
for row in rows:
    team_name = row.find('div', {'class': 'event-cell__name-text'}).text.split(' ')
    if len(team_name[0]) == 3:
        team_id = team_name[0]
    else:
        team_id = team_name[0] + team_name[1][0]
    team_id = team_id.replace('BKN', 'BRK').replace('CHA', 'CHO')
    odd = row.find('span', {'class': 'sportsbook-odds american no-margin default-color'}).text.replace('−', '-')
    
    if idx == 0:
        game['visitor'] = team_id
        game['v_line'] = american_to_decimal(int(odd))
        idx = 1
        
    elif idx == 1:
        game['local'] = team_id
        game['l_line'] = american_to_decimal(int(odd))
        idx = 0
        
        games.append(game)
        game = {'date': today, 'visitor': '', 'local': '', 'v_line': 0, 'l_line': 0}
    
games_df = pd.DataFrame(games)
games_df

,date,visitor,local,v_line,l_line
0,2024-02-23,TOR,ATL,3.450000,1.327869
1,2024-02-23,PHO,HOU,1.555556,2.500000
2,2024-02-23,WAS,OKC,10.000000,1.062500
3,2024-02-23,LAC,MEM,1.243902,4.300000
4,2024-02-23,MIA,NOP,2.240000,1.675676
5,2024-02-23,MIL,MIN,2.500000,1.555556
6,2024-02-23,DEN,POR,1.235294,4.300000
7,2024-02-23,CHO,GSW,7.000000,1.111111
8,2024-02-23,SAS,LAL,4.600000,1.212766


In [16]:
with pd.ExcelWriter('Predictions/games_odds.xlsx') as writer:  
    games_df.to_excel(writer, sheet_name='Tonight', index=0)
    prev_games.to_excel(writer, sheet_name='Previous', index=0)